<a href="https://colab.research.google.com/github/milioe/Moravec/blob/main/3_ComputerVision3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Which GPU I have
!nvidia-smi

Sat Jul 23 20:53:54 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import zipfile 

!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

# Unzip the download
zip_ref = zipfile.ZipFile("10_food_classes_10_percent.zip", "r")
zip_ref.extractall()
zip_ref.close()

--2022-07-23 20:53:57--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.98.128, 74.125.197.128, 74.125.135.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.98.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M   173MB/s    in 0.9s    

2022-07-23 20:53:59 (173 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



In [ ]:
import os # through system

# Walk through 10_food_classes directory and list number of files
for dirpath, dirnames, filenames in os.walk("10_food_classes_10_percent"):
  print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

There are 2 directories and 0 images in '10_food_classes_10_percent'.
There are 10 directories and 0 images in '10_food_classes_10_percent/train'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/ice_cream'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/sushi'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/hamburger'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/ramen'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/chicken_curry'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/steak'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/pizza'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/chicken_wings'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/fried_rice'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/gr

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SHAPE = (224, 224)
BATCH_SIZE=32

train_dir = "10_food_classes_10_percent/train/"
test_dir = "10_food_classes_10_percent/test/"

train_datagen = ImageDataGenerator(rescale=1/225.)
test_datagen = ImageDataGenerator(rescale=1/225.)


train_data_10_percent = train_datagen.flow_from_directory(train_dir,
                                                       target_size=IMAGE_SHAPE,
                                                       batch_size=BATCH_SIZE,
                                                       class_mode="categorical")


test_data_10_percent = train_datagen.flow_from_directory(test_dir,
                                                       target_size=IMAGE_SHAPE,
                                                       batch_size=BATCH_SIZE,
                                                       class_mode="categorical")

Found 750 images belonging to 10 classes.
Found 2500 images belonging to 10 classes.


Models to use:

* ResNetV2
* EfficientNet

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers

In [ ]:
resnet_url = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/5"

efficient_net_url = "https://tfhub.dev/google/efficientnet/b0/feature-vector/1"

In [ ]:
def create_model(model_url, num_classes=10):
  feature_extractor_layer = hub.KerasLayer(model_url,
                                           trainable=False, # freeze layers
                                           name="feature_extraction_layer",
                                           input_shape=IMAGE_SHAPE+(3,))
  
  model = tf.keras.Sequential([
    feature_extractor_layer,
    layers.Dense(num_classes, activation="softmax", name="output_layer")
  ])

  return model

In [ ]:
# 1. Create model
resnet_model = create_model(resnet_url, num_classes=train_data_10_percent.num_classes)

# 2. Compile model
resnet_model.compile(loss="categorical_crossentropy",
                     optimizer=tf.keras.optimizers.Adam(),
                     metrics=["accuracy"])

In [ ]:
resnet_model.fit(train_data_10_percent,
                 epochs=5,
                 steps_per_epoch=len(train_data_10_percent),
                 validation_data=test_data_10_percent,
                 validation_steps=len(test_data_10_percent))

Epoch 1/5
24/24 [==============================] - 32s 758ms/step - loss: 2.0361 - accuracy: 0.3160 - val_loss: 1.2598 - val_accuracy: 0.5988
Epoch 2/5
24/24 [==============================] - 16s 683ms/step - loss: 0.9501 - accuracy: 0.7160 - val_loss: 0.8905 - val_accuracy: 0.7116
Epoch 3/5
24/24 [==============================] - 16s 681ms/step - loss: 0.6498 - accuracy: 0.8147 - val_loss: 0.7703 - val_accuracy: 0.7416
Epoch 4/5
24/24 [==============================] - 16s 689ms/step - loss: 0.4985 - accuracy: 0.8600 - val_loss: 0.7076 - val_accuracy: 0.7740
Epoch 5/5
24/24 [==============================] - 16s 687ms/step - loss: 0.3932 - accuracy: 0.9133 - val_loss: 0.6878 - val_accuracy: 0.7768


In [ ]:
resnet_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 feature_extraction_layer (K  (None, 2048)             23564800  
 erasLayer)                                                      
                                                                 
 output_layer (Dense)        (None, 10)                20490     
                                                                 
Total params: 23,585,290
Trainable params: 20,490
Non-trainable params: 23,564,800
_________________________________________________________________


In [ ]:
#
# 1. Create model
efficient_model = create_model(efficient_net_url, num_classes=train_data_10_percent.num_classes)

# 2. Compile model
efficient_model.compile(loss="categorical_crossentropy",
                     optimizer=tf.keras.optimizers.Adam(),
                     metrics=["accuracy"])

# 3. Fit Efficient
efficient_model.fit(train_data_10_percent,
                    epochs=5,
                    steps_per_epoch=len(train_data_10_percent),
                    validation_data=test_data_10_percent,
                    validation_steps=len(test_data_10_percent))

Epoch 1/5
24/24 [==============================] - 20s 690ms/step - loss: 1.8217 - accuracy: 0.4280 - val_loss: 1.2951 - val_accuracy: 0.7156
Epoch 2/5
24/24 [==============================] - 15s 639ms/step - loss: 1.0615 - accuracy: 0.7547 - val_loss: 0.8775 - val_accuracy: 0.8108
Epoch 3/5
24/24 [==============================] - 15s 643ms/step - loss: 0.7663 - accuracy: 0.8173 - val_loss: 0.7118 - val_accuracy: 0.8304
Epoch 4/5
24/24 [==============================] - 15s 642ms/step - loss: 0.6192 - accuracy: 0.8573 - val_loss: 0.6256 - val_accuracy: 0.8472
Epoch 5/5
24/24 [==============================] - 15s 636ms/step - loss: 0.5255 - accuracy: 0.8853 - val_loss: 0.5713 - val_accuracy: 0.8552


In [ ]:
import joblib

In [ ]:
joblib.dump(resnet_model, 'resnet.pkl')

INFO:tensorflow:Assets written to: ram://d07f1114-eb41-4eb4-9d3d-aa51d685a937/assets


INFO:tensorflow:Assets written to: ram://d07f1114-eb41-4eb4-9d3d-aa51d685a937/assets


['resnet.pkl']

In [ ]:
joblib.dump(efficient_model, 'efficient.pkl')

INFO:tensorflow:Assets written to: ram://522492f6-bdb4-497a-9450-fdd97a16b539/assets


INFO:tensorflow:Assets written to: ram://522492f6-bdb4-497a-9450-fdd97a16b539/assets


['efficient.pkl']